In [1]:
import h5py
import numpy as np
import os
import glob
from sklearn.preprocessing import MinMaxScaler

In [2]:
data_folder = "./RecordGesture"
file_pattern = os.path.join(data_folder, '*.h5')  # 匹配所有 .h5 文件

label_mapping = {
    'Left': 1,
    'PatPat': 0,
    'Right': 2
}

all_images = []
all_labels = []

file_paths = glob.glob(file_pattern)

scaler = MinMaxScaler()

In [3]:
for file_path in file_paths:
    
    file_name = os.path.basename(file_path)
    category = file_name.split('_')[0]  # 提取類別
    
    
    if category in label_mapping:
        
        with h5py.File(file_path, 'r') as file:
            
            f = file['DS1']  
            data = np.zeros([32, 64, 45], dtype = np.float16)
            data[0:32, 0:32, 0:45] = f[0,0:32,0:32, 0:45]
            data[0:32, 32:65, 0:45] = f[1,0:32,0:32, 0:45]
            
            # normalized
            height, width, frames = data.shape
            data_reshaped = data.reshape(-1, 1)
            data_normalized_reshaped = scaler.fit_transform(data_reshaped)
            data_normalized = data_normalized_reshaped.reshape(height, width, frames)
            data = data_normalized
            
            
            label = label_mapping[category]
            
            
            all_images.append(data)  
            all_labels.append(label)  

In [4]:
all_images = np.array(all_images) 
all_labels = np.array(all_labels)

In [5]:
def save_to_npz(images, labels, npz_file):
    np.savez(npz_file, images=images, labels=labels)

save_to_npz(all_images, all_labels, 'data_for_training.npz')